In [69]:
import polars as pl

In [132]:
cahps_df = pl.read_parquet('../../003_data/002_clean-data/national_cms_dialysis-facility_cahps-data.parquet')
facility_df = pl.read_parquet('../../003_data/002_clean-data/national_cms_dialysis-facility_data.parquet')
ballot_measures_df = pl.read_parquet('../../003_data/001_raw-data/2018-2022_ballot-measure_sub-county_data.parquet')
supp_facility_df = pl.read_parquet('../../003_data/001_raw-data/2013-2023_CHHS_dialysis-facility_data.parquet')


In [133]:
cahps_df

year,provider_number,network,facility_name,address_line_1,address_line_2,city,state,zip_code,county,profit_or_nonprofit,phone_number,chain_owned,chain_organization,ichcahps_date,ichcahps_data_availability_code,linearized_score_of_nephrologists_communication_and_caring,star_rating_of_nephrologists_communication_and_caring,linearized_score_of_quality_of_dialysis_center_care_and_operations,star_rating_of_quality_of_dialysis_center_care_and_operations,linearized_score_of_providing_information_to_patients,star_rating_of_providing_information_to_patients,linearized_score_of_rating_of_the_nephrologist,star_rating_of_the_nephrologist,linearized_score_of_rating_of_the_dialysis_center_staff,star_rating_of_the_dialysis_center_staff,linearized_score_of_rating_of_the_dialysis_facility,star_rating_of_the_dialysis_facility,total_number_of_completed_interviews_from_the_fall_and_spring_surveys,ich_cahps_survey_of_patients_experiences_star_rating,survey_response_rate,ichcahps_survey_response_rate,ich_cahps_quality_of_patient_care_star_rating
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""2024""","""52305""","""17""","""SANTA CLARA VALLEY RENAL CARE …","""2220 MOORPARK AVENUE""",null,"""SAN JOSE""","""CA""","""95128""","""Santa Clara""","""non-profit""","""(408) 885-5730""","""no""","""Independent""","""21OCT2022-14JUL2023""","""1""","""85""","""4""","""85""","""5""","""81""","""4""","""88""","""5""","""88""","""4""","""89""","""3""","""47""","""4""","""19""",null,null
"""2024""","""52311""","""18""","""St. Joseph Hospital Renal Cent…","""Sr. Elizabeth Bldg. 1100 W. St…",null,"""Orange""","""CA""","""92868""","""Orange""","""non-profit""","""(714) 771-8037""","""no""","""Independent""","""21OCT2022-14JUL2023""","""101""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""2024""","""52321""","""18""","""Childrens Hospital of Los Ange…","""Division of Nephrology (Dialys…",null,"""Los Angeles""","""CA""","""90027""","""Los Angeles""","""non-profit""","""(323) 361-2560""","""no""","""Independent""","""21OCT2022-14JUL2023""","""102""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""2024""","""52323""","""18""","""Kaiser Foundation Hospital Med…","""Dialysis Unit 4700 Sunset Blvd…",null,"""Los Angeles""","""CA""","""90027""","""Los Angeles""","""non-profit""","""(323) 783-5580""","""yes""","""Kaiser Permanente""","""21OCT2022-14JUL2023""","""1""","""76""","""2""","""82""","""4""","""84""","""5""","""82""","""3""","""88""","""4""","""91""","""4""","""52""","""4""","""25""",null,null
"""2024""","""52334""","""18""","""Arrowhead Regional Medical Cen…","""400 N. Pepper Avenue""",null,"""Colton""","""CA""","""92324""","""San Bernardino""","""non-profit""","""(909) 580-3911""","""no""","""Independent""","""21OCT2022-14JUL2023""","""101""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2018""","""552833""","""18""","""U.S. RENAL CARE SOUTH BAY DIAL…","""4450 182ND STREET""","""""","""REDONDO BEACH""","""CA""","""90278""","""LOS ANGELES""","""profit""","""(310) 542-8032""","""yes""","""US RENAL CARE, INC.""","""04/21/2017 - 01/17/2018""","""258""","""not Available""","""not Available""","""not Available""","""not Available""","""not Available""","""not Available""","""not Available""","""not Available""","""not Available""","""not Available""","""not Available""","""not Available""","""not Available""",null,null,null,"""not Available"""
"""2018""","""552834""","""18""","""DAVITA-VISTA DEL SOL DIALYSIS""","""15002 AMARGOSA ROAD""","""""","""VICTORVILLE""","""CA""","""92394""","""SAN BERNARDINO""","""profit""","""(442) 255-4023""","""yes""","""DAVITA""","""04/21/2017 - 01/17/2018""","""258""","""not Available""","""not Available""","""not Available""","""not Available""","""not Available""","""

In [134]:
# Filtering the supplemental facility information dataframe to include on dialysis clinics

supp_facility_df = supp_facility_df.filter(pl.col("LIC_CAT") == "Chronic Dialysis Clinic")

In [135]:
# Selecting facility ids and geographic features to merge with CMS data
# Stripping extraneous numbers from facility ids to match the id format in CMS data
# Renaming FAC_NO provider_number to match the id format in CMS data

supp_facility_df = (supp_facility_df
 .with_columns(
     pl.col("FAC_NO")
     .alias("provider_number"))
 .with_columns(
     pl.col("COUNTY")
     .alias("county"))
 .with_columns(
     pl.col("FAC_NAME")
     .alias("facility_name"))
 .with_columns(
     pl.col("FAC_ZIP")
     .alias("zip_code"))
 .select([
    "provider_number",
    "facility_name",
    "county",
    "zip_code",
    "ASSEMBLY_DIST",
    "SENATE_DIST",
    "CONGRESS_DIST",
    "CENS_TRACT",
    "LONGITUDE",
    "LATITUDE"
    ])
)

In [124]:
supp_facility_df

provider_number,facility_name,county,FAC_ZIP,ASSEMBLY_DIST,SENATE_DIST,CONGRESS_DIST,CENS_TRACT,LONGITUDE,LATITUDE
str,str,str,str,f64,f64,f64,str,f64,f64
"""306540615""","""ARA KAWAEH DIALYSIS CENTER""","""Tulare""","""93277""",32.0,12.0,20.0,"""06107001902""",-119.311749,36.298925
"""306384020""","""RAI - OCEAN AVENUE - SAN FRANC…","""San Francisco""","""94112""",17.0,11.0,11.0,"""06075031000""",-122.460505,37.724984
"""306374242""","""UCSD DIALYSIS CLINIC""","""San Diego""","""92103""",78.0,39.0,50.0,"""06073000400""",-117.166135,32.75427
"""306364077""","""DESERT CITIES DIALYSIS""","""San Bernardino""","""92395""",34.0,23.0,23.0,"""06071010026""",-117.289977,34.481394
"""306360481""","""RANCHO SAN BERNARDINO DIALYSIS""","""San Bernardino""","""92377""",45.0,29.0,33.0,"""06071002706""",-117.369768,34.137062
…,…,…,…,…,…,…,…,…,…
"""306300213""","""LA PALMA DIALYSIS""","""Orange""","""90623""",65.0,29.0,38.0,"""06059110116""",-118.038842,33.84666
"""306312304""","""RAI - HARDING BLVD. - ROSEVILL…","""Placer""","""95678""",6.0,4.0,4.0,"""06061020806""",-121.268616,38.748799
"""306190407""","""TORRANCE EMERALD DIALYSIS""","""Los Angeles""","""90503""",66.0,26.0,33.0,"""06037650602""",-118.35387,33.84085


In [125]:
merged_df = cahps_df.join(
    supp_facility_df,
    on="facility_name",
    how="left"
)

# Check the shape of the resulting dataframe
print(f"Shape of merged_df: {merged_df.shape}")
print(f"Shape of cahps_df: {cahps_df.shape}")

Shape of merged_df: (7898, 42)
Shape of cahps_df: (5045, 33)


In [136]:
from thefuzz import process, fuzz

# Trying fuzzy merging
def fuzzy_match(x, choices, scorer=fuzz.token_sort_ratio):
    return process.extractOne(x, choices, scorer=scorer)[0]

def fuzzy_merge(df1, df2, key1, key2, threshold=90):
    choices = df2[key2].to_list()
    
    df1 = df1.with_columns([
        pl.col(key1).map_elements(lambda x: fuzzy_match(x, choices)).alias('merge_key'),
        pl.col(key1).map_elements(lambda x: process.extractOne(x, choices, scorer=fuzz.token_sort_ratio)[1]).alias('match_score')
    ])
    
    df1 = df1.filter(pl.col('match_score') >= threshold)
    df1 = df1.drop('match_score')
    
    return df1.join(df2, left_on='merge_key', right_on=key2, how='left')

merged_df_fuzzy = fuzzy_merge(cahps_df, supp_facility_df, 'facility_name', 'facility_name', threshold=85)

# Trying to standardize facility names
def standardize_name(name):
    return name.lower().replace(' ', '').replace('-', '').replace(',', '')

cahps_df = cahps_df.with_columns(pl.col('facility_name').map_elements(standardize_name).alias('std_facility_name'))
supp_facility_df = supp_facility_df.with_columns(pl.col('facility_name').map_elements(standardize_name).alias('std_facility_name'))

merged_df_std = cahps_df.join(
    supp_facility_df,
    on='std_facility_name',
    how='left'
)

# 3. Using multiple columns for matching
merged_df_multi = cahps_df.join(
    supp_facility_df,
    on=['std_facility_name', 'zip_code'],
    how='left'
)

# Comparing approaches
print(f"Original merge shape: {merged_df.shape}")
print(f"Fuzzy merge shape: {merged_df_fuzzy.shape}")
print(f"Standardized name merge shape: {merged_df_std.shape}")
print(f"Multi-column merge shape: {merged_df_multi.shape}")

# Looking at unmatched records
unmatched = cahps_df.join(
    supp_facility_df,
    on='std_facility_name',
    how='left'
).filter(pl.col('LONGITUDE').is_null())

print(f"\nUnmatched records: {unmatched.shape[0]}")
print("\nSample of unmatched facility names:")
print(unmatched.select('facility_name').head(10))

sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.


Original merge shape: (7898, 42)
Fuzzy merge shape: (15254, 43)
Standardized name merge shape: (9609, 44)
Multi-column merge shape: (8791, 43)

Unmatched records: 4163

Sample of unmatched facility names:
shape: (10, 1)
┌─────────────────────────────────┐
│ facility_name                   │
│ ---                             │
│ str                             │
╞═════════════════════════════════╡
│ SANTA CLARA VALLEY RENAL CARE … │
│ St. Joseph Hospital Renal Cent… │
│ Childrens Hospital of Los Ange… │
│ Kaiser Foundation Hospital Med… │
│ Arrowhead Regional Medical Cen… │
│ Kaiser Foundation Hospital Med… │
│ Kaiser Foundation Hospital Med… │
│ Kaiser Foundation Hospital Med… │
│ Kaiser Foundation Hospital Med… │
│ Kaiser Foundation Hospital Med… │
└─────────────────────────────────┘


/var/folders/29/ysfdqqh96xd2zvnqnqd_8fm80000gn/T/ipykernel_77061/2247515017.py:28: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  cahps_df = cahps_df.with_columns(pl.col('facility_name').map_elements(standardize_name).alias('std_facility_name'))
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can 

In [138]:
merged_df_fuzzy

year,provider_number,network,facility_name,address_line_1,address_line_2,city,state,zip_code,county,profit_or_nonprofit,phone_number,chain_owned,chain_organization,ichcahps_date,ichcahps_data_availability_code,linearized_score_of_nephrologists_communication_and_caring,star_rating_of_nephrologists_communication_and_caring,linearized_score_of_quality_of_dialysis_center_care_and_operations,star_rating_of_quality_of_dialysis_center_care_and_operations,linearized_score_of_providing_information_to_patients,star_rating_of_providing_information_to_patients,linearized_score_of_rating_of_the_nephrologist,star_rating_of_the_nephrologist,linearized_score_of_rating_of_the_dialysis_center_staff,star_rating_of_the_dialysis_center_staff,linearized_score_of_rating_of_the_dialysis_facility,star_rating_of_the_dialysis_facility,total_number_of_completed_interviews_from_the_fall_and_spring_surveys,ich_cahps_survey_of_patients_experiences_star_rating,survey_response_rate,ichcahps_survey_response_rate,ich_cahps_quality_of_patient_care_star_rating,merge_key,provider_number_right,county_right,zip_code_right,ASSEMBLY_DIST,SENATE_DIST,CONGRESS_DIST,CENS_TRACT,LONGITUDE,LATITUDE
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,str,f64,f64
"""2024""","""52502""","""18""","""DaVita Kidney Dialysis Care Un…","""3600 E. Martin Luther King Jr.…",null,"""Lynwood""","""CA""","""90262""","""Los Angeles""","""profit""","""(310) 886-5156""","""yes""","""DaVita""","""21OCT2022-14JUL2023""","""1""","""84""","""4""","""88""","""5""","""83""","""4""","""86""","""4""","""92""","""5""","""91""","""4""","""59""","""4""","""24""",null,null,"""KIDNEY DIALYSIS CARE UNIT""","""306190442""","""Los Angeles""","""90262""",62.0,33.0,44.0,"""06037540202""",-118.204095,33.932204
"""2024""","""52502""","""18""","""DaVita Kidney Dialysis Care Un…","""3600 E. Martin Luther King Jr.…",null,"""Lynwood""","""CA""","""90262""","""Los Angeles""","""profit""","""(310) 886-5156""","""yes""","""DaVita""","""21OCT2022-14JUL2023""","""1""","""84""","""4""","""88""","""5""","""83""","""4""","""86""","""4""","""92""","""5""","""91""","""4""","""59""","""4""","""24""",null,null,"""KIDNEY DIALYSIS CARE UNIT""","""306190442""","""Los Angeles""","""90262""",63.0,33.0,44.0,"""06037540202""",-118.204095,33.932204
"""2024""","""52502""","""18""","""DaVita Kidney Dialysis Care Un…","""3600 E. Martin Luther King Jr.…",null,"""Lynwood""","""CA""","""90262""","""Los Angeles""","""profit""","""(310) 886-5156""","""yes""","""DaVita""","""21OCT2022-14JUL2023""","""1""","""84""","""4""","""88""","""5""","""83""","""4""","""86""","""4""","""92""","""5""","""91""","""4""","""59""","""4""","""24""",null,null,"""KIDNEY DIALYSIS CARE UNIT""","""306190442""","""Los Angeles""","""90262""",63.0,33.0,44.0,"""06037540202""",-118.204095,33.932204
"""2024""","""52502""","""18""","""DaVita Kidney Dialysis Care Un…","""3600 E. Martin Luther King Jr.…",null,"""Lynwood""","""CA""","""90262""","""Los Angeles""","""profit""","""(310) 886-5156""","""yes""","""DaVita""","""21OCT2022-14JUL2023""","""1""","""84""","""4""","""88""","""5""","""83""","""4""","""86""","""4""","""92""","""5""","""91""","""4""","""59""","""4""","""24""",null,null,"""KIDNEY DIALYSIS CARE UNIT""","""306190442""","""Los Angeles""","""90262""",63.0,33.0,44.0,"""06037540202""",-118.20389,33.93242
"""2024""","""52502""","""18""","""DaVita Kidney Dialysis Care Un…","""3600 E. Martin Luther King Jr.…",null,"""Lynwood""","""CA""","""90262""","""Los Angeles""","""profit""","""(310) 886-5156""","""yes""","""DaVita""","""21OCT2022-14JUL2023""","""1""","""84""","""4""","""88""","""5""","""83""","""4""","""86""","""4""","""92""","""5""","""91""","""4""","""59""","""4""","""24""",null,null,"""KIDNEY DIALYSIS CARE UNIT""","""306190442""","""Los Angeles""","""90262""",63.0,33.0,44.0,"""06037540202""",-118.20389,33.93242
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,

In [64]:

# It seems like dialysis facilites aren't located in more rural parts of california


# Lowercasing all county names in the dataframes

facility_df = (
    facility_df
    .with_columns(
        pl.col("county")
        .str.to_lowercase()
        .alias("county")
    )
)

ballot_measures_df = (
    ballot_measures_df
    .with_columns(
        pl.col("county")
        .str.to_lowercase()
        .alias("county")
    )
)

cahps_df = (
    cahps_df
    .with_columns(
        pl.col("county")
        .str.to_lowercase()
        .alias("county")
    )
)

# Getting unique counties from all dataframes
ballot_counties = set(ballot_measures_df['county'].unique())
facility_counties = set(facility_df['county'].unique())
cahps_counties = set(cahps_df['county'].unique())

print("Unique counties in ballot_measures_df:")
print(sorted(ballot_counties))

print("\nUnique counties in facility_df:")
print(sorted(facility_counties))

print("\nUnique counties in cahps_df:")
print(sorted(cahps_counties))

# Finding counties that are in one dataframe but not others
counties_only_in_ballot = ballot_counties - facility_counties - cahps_counties
counties_only_in_facility = facility_counties - ballot_counties - cahps_counties
counties_only_in_cahps = cahps_counties - ballot_counties - facility_counties

print("\nCounties only in ballot_measures_df:")
print(sorted(counties_only_in_ballot))

print("\nCounties only in facility_df:")
print(sorted(counties_only_in_facility))

print("\nCounties only in cahps_df:")
print(sorted(counties_only_in_cahps))

# Checking for potential mismatches due to formatting
all_counties = ballot_counties.union(facility_counties).union(cahps_counties)
potential_mismatches = [county for county in all_counties if any(
    county != other_county and county in other_county
    for other_county in all_counties
)]

print("\nPotential county name mismatches:")
print(sorted(potential_mismatches))


Unique counties in ballot_measures_df:
['alameda', 'alpine', 'amador', 'butte', 'calaveras', 'colusa', 'contra costa', 'del norte', 'el dorado', 'fresno', 'glenn', 'humboldt', 'imperial', 'inyo', 'kern', 'kings', 'lake', 'lassen', 'los angeles', 'madera', 'marin', 'mariposa', 'mendocino', 'merced', 'modoc', 'mono', 'monterey', 'napa', 'nevada', 'orange', 'placer', 'plumas', 'riverside', 'sacramento', 'san benito', 'san bernardino', 'san diego', 'san francisco', 'san joaquin', 'san luis obispo', 'san mateo', 'santa barbara', 'santa clara', 'santa cruz', 'shasta', 'sierra', 'siskiyou', 'solano', 'sonoma', 'stanislaus', 'sutter', 'tehama', 'trinity', 'tulare', 'tuolumne', 'ventura', 'yolo', 'yuba']

Unique counties in facility_df:
['', 'alameda', 'butte', 'columbia', 'colusa', 'contra costa', 'del norte', 'el dorado', 'fresno', 'humboldt', 'imperial', 'inyo', 'kern', 'kings', 'lake', 'los angeles', 'madera', 'marin', 'mendocino', 'merced', 'monterey', 'napa', 'nevada', 'orange', 'placer',

In [65]:
supp_facility_df

FAC_NO,FAC_NAME,FAC_CITY,FAC_ZIP,FAC_PHONE,FAC_ADMIN_NAME,FAC_OPERATED_THIS_YR,FAC_OP_PER_BEGIN_DT,FAC_OP_PER_END_DT,FAC_PAR_CORP_NAME,FAC_PAR_CORP_CITY,FAC_PAR_CORP_STATE,FAC_PAR_CORP_ZIP,REPT_PREP_NAME,LICENSE_STATUS,LIC_STATUS_DATE,LIC_ORIG_DATE,REPORT_STATUS,MCAL_PROVIDER_NO,MCARE_PROVIDER_NO,ACLAIMS_NO,ASSEMBLY_DIST,SENATE_DIST,CONGRESS_DIST,CENS_TRACT,MED_SVC_STUDY_AREA,LA_COUNTY_SVC_PLAN_AREA,HEALTH_SVC_AREA,COUNTY,LICENSE_NO,LONGITUDE,LATITUDE,LIC_CAT,LICEE_TOC,TOT_UNDUPLICATED_PATIENTS,TOT_ENCOUNTERS,SURG_OPER_RMS_ON_DEC_31_CNT,…,EQUIP_VAL_10,DT_ACQUIRE_10,MEANS_FOR_ACQUISITION_10,PROJ_OVER_1M,DEPROJ_01,PROJ_EXPENDITURES_01,OSHPD_PROJ_NO_01,DEPROJ_02,PROJ_EXPENDITURES_02,OSHPD_PROJ_NO_02,DEPROJ_03,PROJ_EXPENDITURES_03,OSHPD_PROJ_NO_03,DEPROJ_04,PROJ_EXPENDITURES_04,OSHPD_PROJ_NO_04,DEPROJ_05,PROJ_EXPENDITURES_05,OSHPD_PROJ_NO_05,source_file,year,FAC_STR_ADDR,FAC_PAR_CORP_BUS_ADDR,Description,SUBMITTED_DT,REV_REPT_PREP_NAME,REVISED_DT,CORRECTED_DT,LICENSE_EFF_DATE,LICENSE_EXP_DATE,FACILITY_LEVEL,HCAI_PROJ_NO_01,HCAI_PROJ_NO_02,HCAI_PROJ_NO_03,HCAI_PROJ_NO_04,HCAI_PROJ_NO_05,__index_level_0__
f64,str,str,str,str,str,str,datetime[ns],datetime[ns],str,str,str,str,str,str,datetime[ns],datetime[ns],str,str,str,str,f64,f64,f64,str,str,str,str,str,f64,f64,f64,str,str,f64,f64,f64,…,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,f64,datetime[ns],str,str,str,datetime[ns],datetime[ns],str,f64,f64,f64,f64,f64,i64
3.06540615e8,"""ARA KAWAEH DIALYSIS CENTER""","""VISALIA""","""93277""","""5597419263""","""Teresa Shaw""","""Yes""",2023-01-01 00:00:00,2023-12-31 00:00:00,"""Teresa Shaw""","""Visalia""","""CA - California""","""93277""","""Teresa Shaw""","""Open""",null,null,"""""","""""","""""","""""",32.0,12.0,20.0,"""06107001902""","""233""","""""","""09 - Central""","""Tulare""",5.50006398e8,-119.311749,36.298925,"""Chronic Dialysis Clinic""","""Investor - Limited Liability C…",102.0,1164.0,null,…,null,null,null,"""No""","""""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""file_1""","""23""","""3446 S. MOONEY BLVD""","""3446 S Mooney Blvd""",null,2024-02-12 14:29:00,"""""","""""","""""",2023-06-17 00:00:00,2024-06-16 00:00:00,"""Parent Facility""",null,null,null,null,null,0
3.06197943e8,"""HIGH DESERT HEALTH SYSTEM AMBU…","""LANCASTER""","""93535""","""661 471-4000""","""Lee Dunham""","""Yes""",2023-01-01 00:00:00,2023-12-31 00:00:00,"""""","""""","""""","""""","""LINDA KEITH""","""Open""",null,null,"""""","""""","""""","""""",39.0,23.0,27.0,"""06037900602""","""77.1c""","""Antelope Valley""","""11 - Los Angeles""","""Los Angeles""",6.0000872e7,-118.124835,34.704507,"""Surgical Clinic""","""City or County""",1599.0,1599.0,3.0,…,null,null,null,"""No""","""""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""file_1""","""23""","""335 EAST AVENUE I""","""""",null,2024-02-13 14:26:00,"""""","""""","""""",2023-11-01 00:00:00,2024-10-31 00:00:00,"""Parent Facility""",null,null,null,null,null,1
3.06198064e8,"""MARTIN LUTHER KING, JR. AMBULA…","""LOS ANGELES""","""90059""","""213 699-7102""","""Michael mills""","""Yes""",2023-01-01 00:00:00,2023-12-31 00:00:00,"""""","""""","""""","""""","""LINDA KEITH""","""Open""",null,null,"""""","""""","""""","""""",65.0,35.0,43.0,"""06037540700""","""78.2ooo""","""South""","""11 - Los Angeles""","""Los Angeles""",5.50000727e8,-118.243764,33.923232,"""Surgical Clinic""","""City or County""",3834.0,3834.0,5.0,…,null,null,null,"""No""","""""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""file_1""","""23""","""1670 E 120TH ST""","""""",null,2024-02-13 14:56:00,"""""","""""","""""",2023-05-28 00:00:00,2024-05-27 00:00:00,"""Parent Facility""",null,null,null,null,null,2
3.06197915e8,"""SHRINERS FOR CHILDREN AMBULATO…","""PASADENA""","""91105""","""626-389-9300""","""Kanayo Keri""","""Yes""",2023-01-01 00:00:00,2023-12-31 00:00:00,"""Shriners Hospitals for Childre…","""Tampa""","""FL - F